In [1]:
import pandas as pd
import numpy as np

In [2]:
# pd.set_option('display.float_format', lambda x: '%.5f' % x)
# np.set_printoptions(suppress=True)

In [3]:
url_train = "https://raw.githubusercontent.com/ZeusKwon/data-drive/main/%EC%A3%BC%EC%B0%A8%EC%88%98%EC%9A%94/train.csv"
url_test = "https://raw.githubusercontent.com/ZeusKwon/data-drive/main/%EC%A3%BC%EC%B0%A8%EC%88%98%EC%9A%94/test.csv"
url_submission = "https://raw.githubusercontent.com/ZeusKwon/data-drive/main/%EC%A3%BC%EC%B0%A8%EC%88%98%EC%9A%94/sample_submission.csv"
urlinfo = 'https://raw.githubusercontent.com/ZeusKwon/data-drive/main/%EC%A3%BC%EC%B0%A8%EC%88%98%EC%9A%94/age_gender_info.csv'


train_raw = pd.read_csv(url_train)
test_raw = pd.read_csv(url_test)
submission_raw = pd.read_csv(url_submission)

In [4]:
def engineering_train(df):
    # 칼럼명 변경
    df2 =  df.rename(columns={'단지코드' : 'id', 
                               '총세대수' : 'no_house',
                              '임대건물구분' : 'bd_type',
                              '지역' : 'state',
                              '공급유형' : 'supply_type',
                              '전용면적' : 'sqm',
                              '전용면적별세대수' : 'house_per_sqm',
                              '공가수' : 'no_empty',
                              '자격유형' : 'q_type',
                              '임대보증금' : 'deposit',
                              '임대료' : 'rent',
                              '도보 10분거리 내 지하철역 수(환승노선 수 반영)' : 'subway',
                              '도보 10분거리 내 버스정류장 수' : 'bus',
                              '단지내주차면수' : 'parkinglot',
                              '등록차량수' : 'cars'})
    
    # sqm -> 평수로 변경
    def pyeong(x):
        y = x*0.3025
        return y

    df3 = df2.copy()
    df3['sqp'] = pyeong(df3['sqm']).round()
    df3_columns = ['id', 'no_house', 'bd_type', 'state', 'supply_type', 'sqp',
       'house_per_sqm', 'no_empty', 'q_type', 'deposit', 'rent', 'subway',
       'bus', 'parkinglot', 'cars']
    df3 = df3[df3_columns]
    
    # 데이터 타입 숫자형으로
    df3['deposit'] = pd.to_numeric(df3['deposit'], errors='coerce')
    df3['rent'] = pd.to_numeric(df3['rent'], errors='coerce')
    
    # 지하철 / 버스 결측치 0으로 
    df4 = df3.copy()
    df4['subway'].fillna(0, inplace = True)
    df4['bus'].fillna(0, inplace = True)
    
    # 평수별 차량 보유 대수 계산
    df4['cars2'] = (df4['house_per_sqm'] / df4['no_house']) * df4['cars']
    df4['cars2_per_house'] = df4['cars2'] / df4['house_per_sqm']
    
    # 결측치 처리 (자격유형 A, H, K)
    null = df4[df4['rent'].isnull()]
    null_ahk = null[null['q_type'].isin(['A','H','K'])]
    sqp_dep_mean = df4.groupby(['q_type','sqp'])['deposit'].mean()
    sqp_rent_mean = df4.groupby(['q_type','sqp'])['rent'].mean()

    for i in range(0, len(null_ahk)):
        qt = null_ahk['q_type'].iloc[i]
        sqpn = null_ahk['sqp'].iloc[i]
        null_ahk['deposit'].iloc[i] = sqp_dep_mean.loc[qt, sqpn]
        null_ahk['rent'].iloc[i] = sqp_rent_mean.loc[qt, sqpn]
        
    df4.loc[df4['deposit'].isna(), 'deposit'] = null_ahk['deposit']
    df4.loc[df4['rent'].isna(), 'rent'] = null_ahk['rent']
        
    #결측치 처리 (자격유형 D)
    df4['deposit'] = df4['deposit'].fillna(0)
    df4['rent'] = df4['rent'].fillna(0)
    
    
    # 데이터 오류 행 드랍 (3번 문제)
    # https://www.dacon.io/competitions/official/235745/talkboard/403708?dtype=recent&page=1
    
    error_id_train =  ['C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
    df_train_error = df4[df4.loc[:,'id'].isin(error_id_train) == True]
    df5 = df4.drop(df_train_error.index, axis = 0)
    
    

    
    
    # 분석에 사용 할 칼럼
    df5_columns = ['no_house', 'bd_type', 'state', 'supply_type', 'sqp',
       'house_per_sqm', 'no_empty', 'q_type', 'deposit', 'rent', 'subway',
       'bus', 'parkinglot', 'cars2']
    
    df5 = df5[df5_columns]
    

    
    return df5

In [5]:
train_temp = train_raw.copy()
train_temp =  train_temp.rename(columns={'단지코드' : 'id', 
                           '총세대수' : 'no_house',
                          '임대건물구분' : 'bd_type',
                          '지역' : 'state',
                          '공급유형' : 'supply_type',
                          '전용면적' : 'sqm',
                          '전용면적별세대수' : 'house_per_sqm',
                          '공가수' : 'no_empty',
                          '자격유형' : 'q_type',
                          '임대보증금' : 'deposit',
                          '임대료' : 'rent',
                          '도보 10분거리 내 지하철역 수(환승노선 수 반영)' : 'subway',
                          '도보 10분거리 내 버스정류장 수' : 'bus',
                          '단지내주차면수' : 'parkinglot',
                          '등록차량수' : 'cars'})
def pyeong(x):
    y = x*0.3025
    return y

train_temp['sqp'] = pyeong(train_temp['sqm']).round()
train_temp_columns = ['id', 'no_house', 'bd_type', 'state', 'supply_type', 'sqp',
   'house_per_sqm', 'no_empty', 'q_type', 'deposit', 'rent', 'subway',
   'bus', 'parkinglot', 'cars']
train_temp = train_temp[train_temp_columns]

train_temp['deposit'] = pd.to_numeric(train_temp['deposit'], errors='coerce')
train_temp['rent'] = pd.to_numeric(train_temp['rent'], errors='coerce')

null_temp = train_temp[train_temp['rent'].isnull()]
null_ahk_temp = null_temp[null_temp['q_type'].isin(['A','H','K'])]
sqp_dep_mean_temp = train_temp.groupby(['q_type','sqp'])['deposit'].mean()
sqp_rent_mean_temp = train_temp.groupby(['q_type','sqp'])['rent'].mean()

In [6]:
def engineering_test(df):
    
    df2 =  df.rename(columns={'단지코드' : 'id', 
                               '총세대수' : 'no_house',
                              '임대건물구분' : 'bd_type',
                              '지역' : 'state',
                              '공급유형' : 'supply_type',
                              '전용면적' : 'sqm',
                              '전용면적별세대수' : 'house_per_sqm',
                              '공가수' : 'no_empty',
                              '자격유형' : 'q_type',
                              '임대보증금' : 'deposit',
                              '임대료' : 'rent',
                              '도보 10분거리 내 지하철역 수(환승노선 수 반영)' : 'subway',
                              '도보 10분거리 내 버스정류장 수' : 'bus',
                              '단지내주차면수' : 'parkinglot',
                              })

    # sqm -> 평수로 변경
    def pyeong(x):
        y = x*0.3025
        return y

    df3 = df2.copy()
    df3['sqp'] = pyeong(df3['sqm']).round()
    df3_columns = ['id', 'no_house', 'bd_type', 'state', 'supply_type', 'sqp',
       'house_per_sqm', 'no_empty', 'q_type', 'deposit', 'rent', 'subway',
       'bus', 'parkinglot']
    df3 = df3[df3_columns]

    # 데이터 타입 숫자형으로
    df3['deposit'] = pd.to_numeric(df3['deposit'], errors='coerce')
    df3['rent'] = pd.to_numeric(df3['rent'], errors='coerce')
    
    # 지하철 / 버스 결측치 0으로 
    df4 = df3.copy()
    df4['subway'].fillna(0, inplace = True)
    df4['bus'].fillna(0, inplace = True)
    

    
    
        
    # 결측치 처리 (자격유형 A, H, K)
    
    null = df4[df4['rent'].isnull()]
    null_ahk = null[null['q_type'].isin(['A','H','K'])]

    for i in range(0, len(null_ahk)):
        qt = null_ahk['q_type'].iloc[i]
        sqpn = null_ahk['sqp'].iloc[i]
        null_ahk['deposit'].iloc[i] = sqp_dep_mean_temp.loc[qt, sqpn]
        null_ahk['rent'].iloc[i] = sqp_rent_mean_temp.loc[qt, sqpn]
        
    df4.loc[df4['deposit'].isna(), 'deposit'] = null_ahk['deposit']
    df4.loc[df4['rent'].isna(), 'rent'] = null_ahk['rent']
        
        
    #결측치 처리 (자격유형 D)
    df4['deposit'] = df4['deposit'].fillna(0)
    df4['rent'] = df4['rent'].fillna(0)
    
       # 데이터 오류 행 드랍 (3번 문제)
    # https://www.dacon.io/competitions/official/235745/talkboard/403708?dtype=recent&page=1
#     error_id_test = ['C2335', 'C1327']
#     df_test_error = df4[df4.loc[:,'id'].isin(error_id_test) == True]
#     df5 = df4.drop(df_test_error.index, axis = 0) 
    
    df5 = df4.copy()
    
    # 분석에 사용 할 칼럼
    
    df5_columns = ['id','no_house', 'bd_type', 'state', 'supply_type', 'sqp',
       'house_per_sqm', 'no_empty', 'q_type', 'deposit', 'rent', 'subway',
       'bus', 'parkinglot']
    
    df5 = df5[df5_columns]
    
    return df5

In [7]:
train_eng = engineering_train(train_raw)

from sklearn.model_selection import train_test_split

train, val = train_test_split(train_eng, train_size = 0.8, test_size = 0.2, random_state = 2)

target = 'cars2'
features = train.columns.difference([target], sort = False)

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]


C:\Users\tkryu\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\tkryu\anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [8]:
from category_encoders import CatBoostEncoder
from catboost import CatBoostRegressor
from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

In [9]:
cat_enc = CatBoostEncoder()

X_train_encoded = cat_enc.fit_transform(X_train, y_train)
X_val_encoded = cat_enc.transform(X_val)

cat = CatBoostRegressor(iterations = 1000
                        , random_state = 42
                        , loss_function = 'MAE'
                       )

mae = make_scorer(mean_absolute_error, greater_is_better=False)


grids = {
#     "max_depth": [3,5,7], 
    "learning_rate" : [0.05, 0.07, 0.1]
    }                

fit_params = {"eval_set": [(X_val_encoded, y_val)], 
              "early_stopping_rounds": 50, 
              } 

reg = GridSearchCV(
    cat,
    param_grid=grids,
    cv=5,
    scoring=mae,
    verbose=1,
    n_jobs=-1
)

reg.fit(X_train_encoded, y_train, **fit_params)

print('최적 하이퍼파라미터: ', reg.best_params_)
print('mae_score: ', reg.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
0:	learn: 63.9475053	test: 64.2698720	best: 64.2698720 (0)	total: 128ms	remaining: 2m 8s
1:	learn: 61.4102811	test: 61.5832577	best: 61.5832577 (1)	total: 131ms	remaining: 1m 5s
2:	learn: 58.8725082	test: 59.0714887	best: 59.0714887 (2)	total: 133ms	remaining: 44.2s
3:	learn: 56.8052163	test: 57.0786974	best: 57.0786974 (3)	total: 135ms	remaining: 33.6s
4:	learn: 54.0327483	test: 54.2256348	best: 54.2256348 (4)	total: 137ms	remaining: 27.3s
5:	learn: 51.7670160	test: 51.8985806	best: 51.8985806 (5)	total: 139ms	remaining: 23.1s
6:	learn: 49.9504572	test: 50.0530978	best: 50.0530978 (6)	total: 142ms	remaining: 20.1s
7:	learn: 48.2588599	test: 48.2688115	best: 48.2688115 (7)	total: 144ms	remaining: 17.9s
8:	learn: 46.1896378	test: 46.1743015	best: 46.1743015 (8)	total: 146ms	remaining: 16.1s
9:	learn: 44.3773081	test: 44.1425170	best: 44.1425170 (9)	total: 148ms	remaining: 14.7s
10:	learn: 42.5796976	test: 42.1131249	best: 42.11

135:	learn: 14.6681189	test: 17.0077286	best: 17.0077286 (135)	total: 473ms	remaining: 3.01s
136:	learn: 14.6040098	test: 16.9566305	best: 16.9566305 (136)	total: 476ms	remaining: 3s
137:	learn: 14.5711009	test: 16.9129097	best: 16.9129097 (137)	total: 479ms	remaining: 2.99s
138:	learn: 14.5438469	test: 16.8696157	best: 16.8696157 (138)	total: 483ms	remaining: 2.99s
139:	learn: 14.4699045	test: 16.7771251	best: 16.7771251 (139)	total: 486ms	remaining: 2.98s
140:	learn: 14.3823409	test: 16.7189494	best: 16.7189494 (140)	total: 491ms	remaining: 2.99s
141:	learn: 14.3367311	test: 16.6830794	best: 16.6830794 (141)	total: 493ms	remaining: 2.98s
142:	learn: 14.2641173	test: 16.6654963	best: 16.6654963 (142)	total: 496ms	remaining: 2.97s
143:	learn: 14.2127088	test: 16.6337802	best: 16.6337802 (143)	total: 498ms	remaining: 2.96s
144:	learn: 14.2044698	test: 16.6290073	best: 16.6290073 (144)	total: 500ms	remaining: 2.95s
145:	learn: 14.1890426	test: 16.6157146	best: 16.6157146 (145)	total: 503

290:	learn: 11.0914084	test: 15.0343162	best: 15.0343162 (290)	total: 817ms	remaining: 1.99s
291:	learn: 11.0886582	test: 15.0342952	best: 15.0342952 (291)	total: 820ms	remaining: 1.99s
292:	learn: 11.0837404	test: 15.0287112	best: 15.0287112 (292)	total: 822ms	remaining: 1.98s
293:	learn: 11.0712670	test: 15.0147948	best: 15.0147948 (293)	total: 824ms	remaining: 1.98s
294:	learn: 11.0636926	test: 15.0146264	best: 15.0146264 (294)	total: 826ms	remaining: 1.97s
295:	learn: 11.0622791	test: 15.0147786	best: 15.0146264 (294)	total: 828ms	remaining: 1.97s
296:	learn: 11.0612494	test: 15.0142190	best: 15.0142190 (296)	total: 831ms	remaining: 1.97s
297:	learn: 11.0512246	test: 15.0281503	best: 15.0142190 (296)	total: 833ms	remaining: 1.96s
298:	learn: 11.0460360	test: 15.0273442	best: 15.0142190 (296)	total: 835ms	remaining: 1.96s
299:	learn: 11.0314890	test: 15.0120035	best: 15.0120035 (299)	total: 837ms	remaining: 1.95s
300:	learn: 11.0301246	test: 15.0119069	best: 15.0119069 (300)	total: 

460:	learn: 9.3617067	test: 14.3550443	best: 14.3550443 (460)	total: 1.16s	remaining: 1.36s
461:	learn: 9.3569559	test: 14.3557667	best: 14.3550443 (460)	total: 1.17s	remaining: 1.36s
462:	learn: 9.3434713	test: 14.3590532	best: 14.3550443 (460)	total: 1.17s	remaining: 1.35s
463:	learn: 9.3405108	test: 14.3566197	best: 14.3550443 (460)	total: 1.17s	remaining: 1.35s
464:	learn: 9.3388366	test: 14.3567547	best: 14.3550443 (460)	total: 1.17s	remaining: 1.35s
465:	learn: 9.3386235	test: 14.3566309	best: 14.3550443 (460)	total: 1.17s	remaining: 1.34s
466:	learn: 9.3383061	test: 14.3568579	best: 14.3550443 (460)	total: 1.18s	remaining: 1.34s
467:	learn: 9.3321577	test: 14.3520647	best: 14.3520647 (467)	total: 1.18s	remaining: 1.34s
468:	learn: 9.3300424	test: 14.3526691	best: 14.3520647 (467)	total: 1.18s	remaining: 1.34s
469:	learn: 9.3245614	test: 14.3524464	best: 14.3520647 (467)	total: 1.18s	remaining: 1.33s
470:	learn: 9.3172243	test: 14.3523943	best: 14.3520647 (467)	total: 1.19s	remai

631:	learn: 8.1874567	test: 13.9153732	best: 13.9141017 (630)	total: 1.51s	remaining: 880ms
632:	learn: 8.1867924	test: 13.9156599	best: 13.9141017 (630)	total: 1.51s	remaining: 877ms
633:	learn: 8.1855236	test: 13.9156332	best: 13.9141017 (630)	total: 1.51s	remaining: 875ms
634:	learn: 8.1678874	test: 13.9092134	best: 13.9092134 (634)	total: 1.52s	remaining: 872ms
635:	learn: 8.1642672	test: 13.9090290	best: 13.9090290 (635)	total: 1.52s	remaining: 870ms
636:	learn: 8.1560587	test: 13.9089308	best: 13.9089308 (636)	total: 1.52s	remaining: 867ms
637:	learn: 8.1448905	test: 13.9067329	best: 13.9067329 (637)	total: 1.52s	remaining: 865ms
638:	learn: 8.1412909	test: 13.9070277	best: 13.9067329 (637)	total: 1.53s	remaining: 862ms
639:	learn: 8.1393154	test: 13.9068432	best: 13.9067329 (637)	total: 1.53s	remaining: 860ms
640:	learn: 8.1375999	test: 13.9136478	best: 13.9067329 (637)	total: 1.53s	remaining: 857ms
641:	learn: 8.1309257	test: 13.9187525	best: 13.9067329 (637)	total: 1.53s	remai

800:	learn: 7.3772292	test: 13.7207094	best: 13.6913446 (778)	total: 1.86s	remaining: 462ms
801:	learn: 7.3704101	test: 13.7190940	best: 13.6913446 (778)	total: 1.86s	remaining: 459ms
802:	learn: 7.3668692	test: 13.7197929	best: 13.6913446 (778)	total: 1.86s	remaining: 457ms
803:	learn: 7.3652512	test: 13.7196406	best: 13.6913446 (778)	total: 1.86s	remaining: 454ms
804:	learn: 7.3631840	test: 13.7194436	best: 13.6913446 (778)	total: 1.87s	remaining: 452ms
805:	learn: 7.3608517	test: 13.7170693	best: 13.6913446 (778)	total: 1.87s	remaining: 450ms
806:	learn: 7.3489481	test: 13.6946199	best: 13.6913446 (778)	total: 1.87s	remaining: 447ms
807:	learn: 7.3447261	test: 13.6936863	best: 13.6913446 (778)	total: 1.87s	remaining: 445ms
808:	learn: 7.3372376	test: 13.6918741	best: 13.6913446 (778)	total: 1.88s	remaining: 443ms
809:	learn: 7.3355644	test: 13.6927906	best: 13.6913446 (778)	total: 1.88s	remaining: 440ms
810:	learn: 7.3353739	test: 13.6928080	best: 13.6913446 (778)	total: 1.88s	remai

971:	learn: 6.8307397	test: 13.5559959	best: 13.5432566 (958)	total: 2.21s	remaining: 63.6ms
972:	learn: 6.8279994	test: 13.5548428	best: 13.5432566 (958)	total: 2.21s	remaining: 61.3ms
973:	learn: 6.8099644	test: 13.5311557	best: 13.5311557 (973)	total: 2.21s	remaining: 59ms
974:	learn: 6.8035344	test: 13.5280337	best: 13.5280337 (974)	total: 2.21s	remaining: 56.7ms
975:	learn: 6.8017352	test: 13.5280616	best: 13.5280337 (974)	total: 2.21s	remaining: 54.5ms
976:	learn: 6.8006633	test: 13.5291826	best: 13.5280337 (974)	total: 2.22s	remaining: 52.2ms
977:	learn: 6.7991349	test: 13.5283324	best: 13.5280337 (974)	total: 2.22s	remaining: 49.9ms
978:	learn: 6.7984907	test: 13.5284741	best: 13.5280337 (974)	total: 2.22s	remaining: 47.7ms
979:	learn: 6.7976524	test: 13.5287143	best: 13.5280337 (974)	total: 2.22s	remaining: 45.4ms
980:	learn: 6.7883768	test: 13.5204349	best: 13.5204349 (980)	total: 2.23s	remaining: 43.1ms
981:	learn: 6.7857001	test: 13.5219662	best: 13.5204349 (980)	total: 2.2

In [10]:
y_pred_train = reg.predict(X_train_encoded)
y_pred_val = reg.predict(X_val_encoded)

mae_train = mean_absolute_error(y_train, y_pred_train)
mae_val = mean_absolute_error(y_val, y_pred_val)

print(mae_train)
print(mae_val)

6.767843058796586
13.520402708281429


In [11]:
test = engineering_test(test_raw)

In [12]:
test_mask = submission_raw.code

In [13]:
test_feature = test.columns.difference(['id'], sort = False)

test2 = test[test_feature]


test2_encoded = cat_enc.transform(test2)

In [14]:
y_pred_test = reg.predict(test2_encoded)

In [15]:
test3 = test.copy()
test3['cars'] = y_pred_test

In [16]:
sub = test3.groupby('id')[['cars']].sum()
sub.reset_index(inplace = True)


In [17]:
submission = submission_raw.copy()

for i in range(0,len(submission)):
    n = submission.iloc[i,0]
    submission.iloc[i,1] = sub[sub['id'] == n].iloc[0,1]

submission.set_index('code', drop = True, inplace = True)
submission

,num
code,
C1072,643.268768
C1128,1203.226147
C1456,381.384883
C1840,508.761163
C1332,1145.259853
...,...
C2456,234.409380
C1266,362.413004
C2152,46.955851


In [19]:
# submission.to_csv('hades_submission8.csv', encoding = 'utf-8')

### sub 5 
- catboostregressor
- gridsearchcv : learning rate [0.2, 0.3, 0.4] -> 0.2
- earlystopping : 50 -> iteration 679
- train mae : 1.4556999254502654
- val mae : 13.460767025881024
- **result : 110**


### sub 6
- catboostregressor
- gridsearchcv : learning rate [0.05, 0.1, 0.15, 0.2] -> 0.15
- earlystopping : 50 -> iteration 999 (earlystopping 발동 안됨)
- train mae : 1.1758222016501447
- val mae : 12.532559838766556
- **result : 114.5**

**근소하게나마 train / val mae 차이가 줄었는데 제출점수는 mae가 더 올랐다.**  
**=> 약간의 train/val 차이만으로 과적합을 줄였다고 판단할 수 없다**

### sub 7
- catboostregressor : loss_function(mae) 추가
- gridsearchcv : learning rate [0.1, 0.15, 0.2] -> 0.1
- earlystopping : 50 -> iteration 766
- train mae : 6.6701283058541545
- val mae : 15.069254846402911
- **result : 107.9**

### sub 8
- catboostregressor : loss_function(mae)
- gridsearchcv : learning rate [0.05, 0.07, 0.1] -> 0.07
- earlystopping : 50 -> iteration 989
- train mae : 6.767843058796586
- val mae : 13.520402708281429
- **result : **